## Refresh data

In [6]:
import requests as rq
shots = rq.get("https://api.covid19tracker.ca/reports/province/bc?stat=vaccinations&after=2021-01-01").json()['data']
doubleshots = rq.get("https://api.covid19tracker.ca/reports/province/bc?stat=vaccinated&after=2021-01-01").json()['data']

In [7]:
def cleanser(pair):
    """
    filters out 'None' values, reduces shot&doubleshot down to the desired data point
    """
    (shots_day, doubleshot_day) = pair # unpack tuple
    return {
        doubleshot_day['date']: shots_day['total_vaccinations'] - doubleshot_day['total_vaccinated'] if doubleshot_day['total_vaccinated'] != None else 0
    }

singleshots = map(cleanser, zip(shots, doubleshots)) # number of people who have had at least one shot
listed = list(singleshots) # for easier viewing/debugging
single = listed[-2]

In [8]:
import matplotlib as plt
